In [ ]:
'''!pip install packaging==20.0
!pip install -U spacy
!pip install spacy_langdetect
!pip install tweepy'''

In [ ]:
'''import pickle5 as pickle
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
import tweepy
import pandas'''

In [ ]:
'''with open('COVID19-GR-0122-0605.pkl', 'rb') as f:
    data = pickle.load(f)
id_list = data.values.tolist()
id_list'''

In [ ]:
'''def get_lang_detector(nlp, name):
    return LanguageDetector()
nlp = spacy.load('en_core_web_md')
Language.factory('language_detector', func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)'''

In [ ]:
'''bearer_token = 'AAAAAAAAAAAAAAAAAAAAACi4aAEAAAAAWsn9pw2qDs9%2BtobId%2F3Cdrt5ChE%3DYuDkoc55cbdwMmcxKw0MEjk1hvMsofYm7yO5eFmam8bEkOoLC6'
consumer_key = 'IRlxi73sbYhJae1umpgPbbFHz'
consumer_secret = 'yLDz9hfaYQy7Y15CSpmQ4WwOGd06oXrm4x9UcGDvPUKRRIECzv'
access_token = '1502949802727940099-RoeyynIyOnf7EsqRgJWbcBrP5fad4M'
access_token_secret = 'KS6W4S51FmprD0wfaWj6RpAn7i3qxaScY6L7GgpLUUon6'

client = tweepy.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_token, access_token_secret=access_token_secret)
texts = []
for i in range(100,20000,100):
    tweets = client.get_tweets(id_list[i-100:i])
    for j in range(len(tweets.data)):
        tweets.data[j] = str(tweets.data[j])
        tweets.data[j] = tweets.data[j].replace('\n', ' ')
        doc = nlp(tweets.data[j])
        if doc._.language['language'] == 'en' and doc._.language['score'] >= 0.9:
            texts.append(tweets.data[j])

df = pandas.DataFrame(texts, columns=['text'])
df
#df.to_csv('tweets.csv', sep='\t', index=False)'''

In [ ]:
'''df = pandas.read_csv('tweets.csv', sep='\t')
tweets = df.text.tolist()
questions = []
for tweet in tweets:
    doc = nlp(tweet)
    for sent in doc.sents:
        for token in sent:
            if token.shape_ == '?':
                questions.append(str(sent).strip().lower())
questions = list(dict.fromkeys(questions))
questions = {'text': questions}
df = pandas.DataFrame(questions)
df
#df.to_csv('questions.csv', '\t', index=False)'''

In [ ]:
!pip install -U spacy
!pip install rasa_nlu
!pip install scipy
!pip install gensim

In [ ]:
!pip install packaging==20.0

In [1]:
from rasa_nlu.training_data  import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer, Interpreter
from rasa_nlu import config
from gensim import corpora, models
from gensim.models import ldamodel
import spacy
import pandas

In [ ]:
spacy.cli.download("en_core_web_md")
nlp = spacy.load("en_core_web_md")

In [ ]:
df = pandas.read_csv('questions.csv', sep='\t')
questions = df.text.tolist()

In [ ]:
keywords = [[] for item in range(len(questions))]

In [ ]:
pos = ['NOUN', 'ADJ', 'VERB', 'ADV', 'PRON']
for question in questions:
  doc = nlp(question)
  for token in doc:
    if token.pos_ in pos:
      keywords[questions.index(question)].append(str(token))
keywords

In [ ]:
dictionary = corpora.Dictionary(keywords)
corpus = [dictionary.doc2bow(word) for word in keywords]
lda = ldamodel.LdaModel(corpus, num_topics=5, random_state=1, iterations=300, passes=10)

In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
vis

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
training_data = load_data('training_data.json')
trainer = Trainer(config.load('config.yaml'))
trainer.train(training_data)
model_directory = trainer.persist('/content/')
interpreter = Interpreter.load(model_directory)

In [ ]:
intents = []
confidence = []
for question in questions:
  intents.append(interpreter.parse(question).get('intent').get('name'))
  confidence.append(interpreter.parse(question).get('intent').get('confidence'))
df['intent'] = intents
df['confidence'] = confidence
df

In [ ]:
df.drop(df[(df.intent == 'coronavirus spread') & (df.confidence >=0.33) & (df.confidence <= 0.43)].index, inplace=True)
df

In [ ]:
df.to_csv('questions.csv', sep='\t', columns=['text'], index=False)